# Nauka pyTorch poprzez trening i Ocena Modeli na Zbiorze Danych Caltech101
Ten notebook ma na celu przeprowadzenie treningu modelu niestandardowego i modelu ResNet-18 na zbiorze danych Caltech101 oraz ich ocenę. Ponadto, zostanie utworzony interfejs Gradio do interakcji z modelem ResNet-18.

# Importowanie bibliotek
Najpierw zaimportujemy wszystkie niezbędne biblioteki do przetwarzania obrazów, trenowania modeli, ładowania danych oraz interakcji z modelem za pomocą Gradio.

In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Caltech101
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import gradio as gr


# Definiowanie transformacji i ładowanie zbioru danych
## Transformacje
Definiujemy transformacje, które będą stosowane do obrazów wejściowych. Transformacje obejmują:

- Resize: Zmiana rozmiaru obrazów do 224x224 pikseli.
- Grayscale: Konwersja obrazów na 3-kanałowe obrazy w odcieniach szarości.
- ToTensor: Konwersja obrazów do formatu tensora.
- Normalize: Normalizacja obrazów z użyciem określonych średnich i odchyleń standardowych.
## Ładowanie zbioru danych
Ładujemy zbiór danych Caltech101 i dzielimy go na zestawy treningowe i walidacyjne w stosunku 80:20.

In [53]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale images to 3 channels
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load the dataset
dataset = Caltech101(root='./data', download=True, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified


# Definiowanie niestandardowego modelu CNN
Tworzymy niestandardowy model CNN, który będzie wykorzystywany do klasyfikacji obrazów. Model ten składa się z:

- Części ekstrakcji cech: Dwie warstwy konwolucyjne, każda z warstwą aktywacji ReLU i warstwą MaxPool.
- Części klasyfikacji: Dwie warstwy w pełni połączone z warstwami Dropout i ReLU.
python


In [54]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128 * 56 * 56, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Ładowanie modelu ResNet-18
Załadujemy wstępnie wytrenowany model ResNet-18 i zmodyfikujemy jego ostatnią w pełni połączoną warstwę, aby dopasować ją do liczby klas w zbiorze danych Caltech101.

In [55]:
# Load the pre-trained ResNet-18 model
resnet_model = resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes in Caltech101
num_classes = len(dataset.categories)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)


/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Inicjalizacja niestandardowego modelu
Inicjalizujemy niestandardowy model z odpowiednią liczbą kla

In [ ]:
# Initialize the custom model
custom_model = CustomCNN(num_classes=num_classes)

# Definiowanie funkcji straty i optymalizatorów
Zdefiniujemy funkcję straty (CrossEntropyLoss) oraz optymalizatory (SGD) dla obu modeli.

In [ ]:
# Define loss function and optimizer for both models
criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)
custom_optimizer = optim.SGD(custom_model.parameters(), lr=0.001, momentum=0.9)

# Funkcja pętli trenowania
Definiujemy funkcję pętli trenowania, która będzie:

- Trenować model na zbiorze treningowym.
- Obliczać stratę i dokładność na zbiorze treningowym.
- Walidować model na zbiorze walidacyjnym.
- Obliczać stratę i dokładność na zbiorze walidacyjnym.
- Wyświetlać wyniki dla każdej epoki.

In [57]:
# Training loop function
def train_model(model, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss /= len(val_loader)

        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")




# Trenowanie obu modeli
Teraz przeprowadzimy trening obu modeli: niestandardowego modelu CNN oraz zmodyfikowanego ResNet-18.

In [58]:
# Train both models
print("Training Custom Model:")
train_model(custom_model, custom_optimizer)

# Save the trained model
torch.save(resnet_model.state_dict(), 'caltech101_resnet18.pth')



Training Custom Model:
Epoch [1/5], Train Loss: 3.7045, Train Accuracy: 24.55%, Validation Loss: 3.2122, Validation Accuracy: 31.39%
Epoch [2/5], Train Loss: 3.2525, Train Accuracy: 31.03%, Validation Loss: 2.9720, Validation Accuracy: 37.44%
Epoch [3/5], Train Loss: 2.9953, Train Accuracy: 34.99%, Validation Loss: 2.7649, Validation Accuracy: 41.42%
Epoch [4/5], Train Loss: 2.7763, Train Accuracy: 38.55%, Validation Loss: 2.5976, Validation Accuracy: 44.41%
Epoch [5/5], Train Loss: 2.5172, Train Accuracy: 42.67%, Validation Loss: 2.4280, Validation Accuracy: 47.12%


In [ ]:
print("\nTraining ResNet-18 Model:")
train_model(resnet_model, resnet_optimizer)
# Save the trained model
torch.save(resnet_model.state_dict(), 'caltech101_resnet18.pth')


# Ładowanie modelu
Ładujemy zapisany model ResNet-18 do stanu ewaluacji.

In [ ]:
# Load the saved model for inference
model.load_state_dict(torch.load('caltech101_resnet18.pth'))
model.eval()


# Tworzenie interfejsu Gradio do interakcji z modelem
Ładujemy zapisany model ResNet-18, definiujemy funkcję predykcji i tworzymy interfejs Gradio.


In [49]:
# Define the prediction function
def predict(im):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.Grayscale(num_output_channels=3),  # Convert grayscale images to 3 channels
        transforms.ToTensor(),  # Convert images to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])
    im = transform(im).unsqueeze(0)  # Transform and add batch dimension
    with torch.no_grad():
        output = model(im)  # Forward pass
    output = torch.nn.functional.softmax(output[0], dim=0)  # Apply softmax
    return {train_dataset.categories[i]: float(output[i]) for i in range(len(output))}


# Create Gradio interface
imagein = gr.Image(type='pil')
label = gr.Label(num_top_classes=5)

gr.Interface(fn=predict, inputs=imagein, outputs=label).launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
